In [ ]:
from psyclone.parse.algorithm import parse
_, inv_info = parse("shallow_alg.f90", api="gocean1.0")

In [ ]:
from psyclone.psyGen import PSyFactory, TransInfo
psy = PSyFactory("gocean1.0", distributed_memory=False).create(inv_info)

In [ ]:
print(psy.gen)

In [ ]:
print(psy.invokes.names)
sched = psy.invokes.get('invoke_0').schedule
sched.view()


In [ ]:
trans_info = TransInfo()
print(trans_info.list)
fuse_trans = trans_info.get_trans_name('LoopFuseTrans')
omp_trans = trans_info.get_trans_name('GOceanOMPParallelLoopTrans')


In [ ]:
# invoke0
# fuse all outer loops
lf1_sched, _ = fuse_trans.apply(sched[0], sched[1])
lf2_sched, _ = fuse_trans.apply(lf1_sched[0], lf1_sched[1])
lf3_sched, _ = fuse_trans.apply(lf2_sched[0], lf2_sched[1])
lf3_sched.view()


In [ ]:
# fuse all inner loops
lf4_sched, _ = fuse_trans.apply(lf3_sched[0].loop_body[0], lf3_sched[0].loop_body[1])
lf5_sched, _ = fuse_trans.apply(lf4_sched[0].loop_body[0], lf4_sched[0].loop_body[1])
lf6_sched, _ = fuse_trans.apply(lf5_sched[0].loop_body[0], lf5_sched[0].loop_body[1])
lf6_sched.view()


In [ ]:
ol_sched, _ = omp_trans.apply(lf6_sched[0])
ol_sched.view()


In [ ]:
print(psy.gen)